In [55]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj,subgraph,remove_isolated_nodes
from torch_geometric.data import Data
import random
dataset = Planetoid(root='./data', name='Citeseer')

In [81]:
data = dataset[0]
a = [i for i in range(1000)]
indices = random.sample(a,100)
indices,index_of_index = torch.unique(torch.LongTensor(indices),return_inverse=True)
indices = indices.tolist()
index_of_index = index_of_index.tolist()
map = {}
for i in range(len(indices)):
    map[indices[i]] = index_of_index[i]
print(map)
# data.x[indices]

{1: 72, 11: 75, 19: 71, 24: 88, 54: 57, 61: 26, 70: 31, 97: 90, 116: 65, 120: 63, 123: 99, 147: 45, 155: 42, 166: 16, 192: 14, 195: 8, 196: 78, 201: 53, 215: 59, 246: 60, 258: 5, 269: 76, 301: 25, 302: 10, 303: 62, 317: 93, 347: 97, 354: 66, 360: 30, 361: 85, 373: 89, 374: 29, 383: 23, 389: 36, 398: 48, 408: 79, 410: 28, 422: 49, 426: 55, 432: 84, 443: 12, 486: 9, 492: 22, 509: 81, 516: 87, 522: 46, 526: 67, 537: 77, 538: 19, 541: 61, 554: 74, 555: 98, 587: 70, 602: 4, 626: 13, 627: 24, 630: 2, 631: 17, 640: 44, 642: 3, 644: 56, 646: 11, 647: 40, 648: 50, 684: 96, 685: 38, 688: 37, 689: 80, 697: 92, 702: 41, 708: 39, 715: 35, 729: 43, 739: 83, 754: 6, 755: 0, 765: 18, 774: 58, 779: 51, 793: 86, 799: 91, 802: 82, 809: 54, 834: 32, 850: 47, 852: 94, 858: 52, 872: 21, 875: 95, 898: 7, 904: 73, 907: 33, 921: 27, 941: 20, 949: 1, 950: 15, 952: 64, 959: 34, 974: 69, 985: 68}


In [95]:
sub_edge_x = data.x[indices]
sub_edge_index,sub_edge_attr = subgraph(indices,edge_index=data.edge_index,edge_attr=data.edge_attr)
sub_edge_index = sub_edge_index.tolist()
for i in range(len(sub_edge_index[0])):
    sub_edge_index[0][i] = map[sub_edge_index[0][i]]
    sub_edge_index[1][i] = map[sub_edge_index[1][i]]

edge_index,edge_attr,no_mask = remove_isolated_nodes(torch.LongTensor(sub_edge_index),num_nodes=100)
sub_x = sub_edge_x[no_mask]
sb_graph = Data(x=sub_x,edge_index=edge_index,edge_attr=edge_attr)
print(sb_graph.edge_index)
print(sub_edge_index)
print(to_dense_adj(edge_index).shape)
print(sb_graph.x.shape)

[[72, 9, 9, 22, 81, 61, 74, 24, 0, 0, 91, 54], [9, 72, 81, 0, 9, 54, 91, 0, 22, 24, 74, 61]]
tensor([[6, 1, 1, 2, 8, 5, 7, 3, 0, 0, 9, 4],
        [1, 6, 8, 0, 1, 4, 9, 0, 2, 3, 7, 5]])
[[72, 9, 9, 22, 81, 61, 74, 24, 0, 0, 91, 54], [9, 72, 81, 0, 9, 54, 91, 0, 22, 24, 74, 61]]
torch.Size([1, 10, 10])
torch.Size([10, 3703])


In [14]:
# distence_coefficient = []
# for i in range(data.x.shape[0]):
#     loss = F.mse_loss(data.x[i],data.x,reduction='none')
#     loss = loss.sum(dim=1)
#     loss = loss.negative().exp()
#     distence_coefficient.append(loss.tolist())
# distence_coefficient
# distence_coefficient = torch.Tensor(distence_coefficient)
# distence_coefficient.shape

distences = F.mse_loss(data.x.unsqueeze(dim=1),data.x,reduction='none')
distences.detach()
distences = distences.sum(dim=2)
distence_coefficient = distences.negative().exp()

dense_adj = to_dense_adj(data.edge_index).squeeze(0)

masked_distence_coefficient = distence_coefficient.mul(dense_adj)

/tmp/ipykernel_2291/585090548.py:11: UserWarning: Using a target size (torch.Size([3327, 3703])) that is different to the input size (torch.Size([100, 1, 3703])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  distences = F.mse_loss(data.x[0:100].unsqueeze(dim=1),data.x,reduction='none')


RuntimeError: The size of tensor a (100) must match the size of tensor b (3327) at non-singleton dimension 0

In [6]:
distences = F.mse_loss(data.x[0],data.x,reduction='none')
distences.detach()
distences = distences.sum(dim=1)
distences

/tmp/ipykernel_30401/1841309735.py:1: UserWarning: Using a target size (torch.Size([3327, 3703])) that is different to the input size (torch.Size([3703])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  distences = F.mse_loss(data.x[0],data.x,reduction='none')


tensor([ 0., 60., 54.,  ..., 63., 63., 53.])

In [7]:
distences = F.mse_loss(data.x[0:10].unsqueeze(dim=1),data.x,reduction='none')
distences.detach()
distences = distences.sum(dim=2)
distences

/tmp/ipykernel_30401/2889968087.py:1: UserWarning: Using a target size (torch.Size([3327, 3703])) that is different to the input size (torch.Size([10, 1, 3703])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  distences = F.mse_loss(data.x[0:10].unsqueeze(dim=1),data.x,reduction='none')


tensor([[ 0., 60., 54.,  ..., 63., 63., 53.],
        [60.,  0., 58.,  ..., 69., 65., 57.],
        [54., 58.,  0.,  ..., 65., 57., 39.],
        ...,
        [58., 64., 56.,  ..., 71., 69., 59.],
        [58., 58., 50.,  ..., 65., 61., 53.],
        [65., 69., 53.,  ..., 74., 66., 54.]])

In [9]:
F.mse_loss(data.x[0],data.x[2],reduction='sum')

tensor(54.)

In [11]:
((data.x[0] - data.x[2])**2).sum()

tensor(54.)